# 基于截断策略的机器阅读理解任务实现（滑动窗口版本）

## Step1 导入相关包

In [2]:
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer, DefaultDataCollator

## Step2 数据集加载

In [3]:
# 如果可以联网，直接使用load_dataset进行加载
#datasets = load_dataset("cmrc2018", cache_dir="data")
# 如果无法联网，则使用下面的方式加载数据集
datasets = DatasetDict.load_from_disk("mrc_data")
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 10142
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 3219
    })
    test: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 1002
    })
})

In [4]:
datasets["train"][0]

{'id': 'TRAIN_186_QUERY_0',
 'context': '范廷颂枢机（，），圣名保禄·若瑟（），是越南罗马天主教枢机。1963年被任为主教；1990年被擢升为天主教河内总教区宗座署理；1994年被擢升为总主教，同年年底被擢升为枢机；2009年2月离世。范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生；童年时接受良好教育后，被一位越南神父带到河内继续其学业。范廷颂于1940年在河内大修道院完成神学学业。范廷颂于1949年6月6日在河内的主教座堂晋铎；及后被派到圣女小德兰孤儿院服务。1950年代，范廷颂在河内堂区创建移民接待中心以收容到河内避战的难民。1954年，法越战争结束，越南民主共和国建都河内，当时很多天主教神职人员逃至越南的南方，但范廷颂仍然留在河内。翌年管理圣若望小修院；惟在1960年因捍卫修院的自由、自治及拒绝政府在修院设政治课的要求而被捕。1963年4月5日，教宗任命范廷颂为天主教北宁教区主教，同年8月15日就任；其牧铭为「我信天主的爱」。由于范廷颂被越南政府软禁差不多30年，因此他无法到所属堂区进行牧灵工作而专注研读等工作。范廷颂除了面对战争、贫困、被当局迫害天主教会等问题外，也秘密恢复修院、创建女修会团体等。1990年，教宗若望保禄二世在同年6月18日擢升范廷颂为天主教河内总教区宗座署理以填补该教区总主教的空缺。1994年3月23日，范廷颂被教宗若望保禄二世擢升为天主教河内总教区总主教并兼天主教谅山教区宗座署理；同年11月26日，若望保禄二世擢升范廷颂为枢机。范廷颂在1995年至2001年期间出任天主教越南主教团主席。2003年4月26日，教宗若望保禄二世任命天主教谅山教区兼天主教高平教区吴光杰主教为天主教河内总教区署理主教；及至2005年2月19日，范廷颂因获批辞去总主教职务而荣休；吴光杰同日真除天主教河内总教区总主教职务。范廷颂于2009年2月22日清晨在河内离世，享年89岁；其葬礼于同月26日上午在天主教河内总教区总主教座堂举行。',
 'question': '范廷颂是什么时候被任为主教的？',
 'answers': {'text': ['1963年'], 'answer_start': [30]}}

## Step3 数据预处理

In [5]:
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")
#tokenizer

In [6]:
sample_dataset = datasets["train"].select(range(10))

In [78]:
tokenized_examples = tokenizer(text=sample_dataset["question"],

                               text_pair=sample_dataset["context"],
                               return_offsets_mapping=True,     #返回每个token在原始文本中的字符位置
                               return_overflowing_tokens=True,  #启用滑动窗口。当文本超过max_length时，不直接截断，而是生成多个首尾重叠的文本片段
                               stride=128,  #相邻窗口之间重叠的token数量（滑动窗口的步长）
                               max_length=384,  #每个窗口最大长度是384，即每个窗口只有384个token，但是可能由于英文的出现，使得字符数量大于384
                               truncation="only_second", padding="max_length")
tokenized_examples.keys()
'''⭐⭐⭐⭐⭐
取dataset中的前10个样本组(每个样本组里面内容如datasets["train"][0]输出所示)并token化成 格式为如输出结果所示的tokenized_examples，每个列表（例如input_ids、overflow_to_sample_mapping）里面包含10个数据各自对应的input_ids 和 overflow_to_sample_mapping
例如：'input_ids': [
        [101, 123, 456, ...],  # 窗口0的token IDs
        [101, 789, 101, ...],  # 窗口1的token IDs
        [101, 234, 567, ...],  # 窗口2的token IDs
    # 关键映射关系⭐⭐⭐⭐⭐
        'overflow_to_sample_mapping': [0, 0, 0, 1, 1, 2, ...]
        #得出的数字不是窗口编号，而是原始样本的索引
            第0个特征（窗口）由第0个样本生成
            第1个特征（窗口）由第0个样本生成
            第2个特征（窗口）由第0个样本生成
            第3个特征（窗口）由第1个样本生成
            ...


overflow_to_sample_mapping 是一个列表，其中每个元素表示当前的特征（窗口）是由哪个原始样本生成的

# 假设原始第一个样本（样本0）的上下文有600个token，max_length=384
原始序列: [token0, token1, token2, ..., token599]
# 启用滑动窗口后生成：一个样本对应多个窗口⭐⭐⭐⭐⭐
窗口1: [token0 到 token383]    # 第1-384个token
窗口2: [token256 到 token511]  # 从256开始，重叠128个token
窗口3: [token384 到 token599]  # 从384开始，可能不足384个token会填充
'''

'\n取dataset中的前10个样本组(每个样本组里面内容如datasets["train"][0]输出所示)并token化成 格式为如输出结果所示的tokenized_examples，每个列表（例如input_ids、overflow_to_sample_mapping）里面包含10个数据各自对应的input_ids 和 overflow_to_sample_mapping\n例如：\'input_ids\': [\n        [101, 123, 456, ...],  # 窗口0的token IDs\n        [101, 789, 101, ...],  # 窗口1的token IDs\n        [101, 234, 567, ...],  # 窗口2的token IDs\n    # 关键映射关系\n        \'overflow_to_sample_mapping\': [0, 0, 0, 1, 1, 2, ...]\n        #得出的数字不是窗口编号，而是原始样本的索引\n            第0个特征（窗口）由第0个样本生成\n            第1个特征（窗口）由第0个样本生成\n            第2个特征（窗口）由第0个样本生成\n            第3个特征（窗口）由第1个样本生成\n            ...\n\n\noverflow_to_sample_mapping 是一个列表，其中每个元素表示当前的特征（窗口）是由哪个原始样本生成的\n\n# 假设原始第一个样本（样本0）的上下文有600个token，max_length=384\n原始序列: [token0, token1, token2, ..., token599]\n# 启用滑动窗口后生成：一个样本对应多个窗口\n窗口1: [token0 到 token383]    # 第1-384个token\n窗口2: [token256 到 token511]  # 从256开始，重叠128个token\n窗口3: [token384 到 token599]  # 从384开始，可能不足384个token会填充\n'

In [79]:
tokenized_examples["overflow_to_sample_mapping"], len(tokenized_examples["overflow_to_sample_mapping"])
#len的输出为：当前处理的10个样本经过tokenization和滑动窗口处理后生成的总窗口数

([0,
  0,
  0,
  1,
  1,
  1,
  2,
  2,
  2,
  3,
  3,
  3,
  4,
  4,
  4,
  5,
  5,
  5,
  6,
  6,
  6,
  7,
  7,
  7,
  8,
  8,
  8,
  9,
  9],
 29)

In [80]:
#遍历前3个窗口，看看tokenization是否正确
for sen in tokenizer.batch_decode(tokenized_examples["input_ids"][:3]):     #遍历解码后的前3个窗口,打印每个窗口的文本内容
    print(sen)

[CLS] 范 廷 颂 是 什 么 时 候 被 任 为 主 教 的 ？ [SEP] 范 廷 颂 枢 机 （ ， ） ， 圣 名 保 禄 · 若 瑟 （ ） ， 是 越 南 罗 马 天 主 教 枢 机 。 1963 年 被 任 为 主 教 ； 1990 年 被 擢 升 为 天 主 教 河 内 总 教 区 宗 座 署 理 ； 1994 年 被 擢 升 为 总 主 教 ， 同 年 年 底 被 擢 升 为 枢 机 ； 2009 年 2 月 离 世 。 范 廷 颂 于 1919 年 6 月 15 日 在 越 南 宁 平 省 天 主 教 发 艳 教 区 出 生 ； 童 年 时 接 受 良 好 教 育 后 ， 被 一 位 越 南 神 父 带 到 河 内 继 续 其 学 业 。 范 廷 颂 于 1940 年 在 河 内 大 修 道 院 完 成 神 学 学 业 。 范 廷 颂 于 1949 年 6 月 6 日 在 河 内 的 主 教 座 堂 晋 铎 ； 及 后 被 派 到 圣 女 小 德 兰 孤 儿 院 服 务 。 1950 年 代 ， 范 廷 颂 在 河 内 堂 区 创 建 移 民 接 待 中 心 以 收 容 到 河 内 避 战 的 难 民 。 1954 年 ， 法 越 战 争 结 束 ， 越 南 民 主 共 和 国 建 都 河 内 ， 当 时 很 多 天 主 教 神 职 人 员 逃 至 越 南 的 南 方 ， 但 范 廷 颂 仍 然 留 在 河 内 。 翌 年 管 理 圣 若 望 小 修 院 ； 惟 在 1960 年 因 捍 卫 修 院 的 自 由 、 自 治 及 拒 绝 政 府 在 修 院 设 政 治 课 的 要 求 而 被 捕 。 1963 年 4 月 5 日 ， 教 宗 任 命 范 廷 颂 为 天 主 教 北 宁 教 区 主 教 ， 同 年 8 月 15 日 就 任 ； 其 牧 铭 为 「 我 信 [SEP]
[CLS] 范 廷 颂 是 什 么 时 候 被 任 为 主 教 的 ？ [SEP] 越 南 民 主 共 和 国 建 都 河 内 ， 当 时 很 多 天 主 教 神 职 人 员 逃 至 越 南 的 南 方 ， 但 范 廷 颂 仍 然 留 在 河 内 。 翌 年 管 理 圣 若 望 小 修 院 ； 惟 在 1960 年 因 捍 卫 修 院 的 自 由 、 自 治 及 拒 

In [81]:
print(tokenized_examples["offset_mapping"][:3])

[[(0, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (0, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (15, 16), (16, 17), (17, 18), (18, 19), (19, 20), (20, 21), (21, 22), (22, 23), (23, 24), (24, 25), (25, 26), (26, 27), (27, 28), (28, 29), (29, 30), (30, 34), (34, 35), (35, 36), (36, 37), (37, 38), (38, 39), (39, 40), (40, 41), (41, 45), (45, 46), (46, 47), (47, 48), (48, 49), (49, 50), (50, 51), (51, 52), (52, 53), (53, 54), (54, 55), (55, 56), (56, 57), (57, 58), (58, 59), (59, 60), (60, 61), (61, 62), (62, 63), (63, 67), (67, 68), (68, 69), (69, 70), (70, 71), (71, 72), (72, 73), (73, 74), (74, 75), (75, 76), (76, 77), (77, 78), (78, 79), (79, 80), (80, 81), (81, 82), (82, 83), (83, 84), (84, 85), (85, 86), (86, 87), (87, 91), (91, 92), (92, 93), (93, 94), (94, 95), (95, 96), (96, 97), (97, 98), (98, 99), 

In [82]:
print(tokenized_examples["offset_mapping"][0], len(tokenized_examples["offset_mapping"][0]))

[(0, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (0, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (15, 16), (16, 17), (17, 18), (18, 19), (19, 20), (20, 21), (21, 22), (22, 23), (23, 24), (24, 25), (25, 26), (26, 27), (27, 28), (28, 29), (29, 30), (30, 34), (34, 35), (35, 36), (36, 37), (37, 38), (38, 39), (39, 40), (40, 41), (41, 45), (45, 46), (46, 47), (47, 48), (48, 49), (49, 50), (50, 51), (51, 52), (52, 53), (53, 54), (54, 55), (55, 56), (56, 57), (57, 58), (58, 59), (59, 60), (60, 61), (61, 62), (62, 63), (63, 67), (67, 68), (68, 69), (69, 70), (70, 71), (71, 72), (72, 73), (73, 74), (74, 75), (75, 76), (76, 77), (77, 78), (78, 79), (79, 80), (80, 81), (81, 82), (82, 83), (83, 84), (84, 85), (85, 86), (86, 87), (87, 91), (91, 92), (92, 93), (93, 94), (94, 95), (95, 96), (96, 97), (97, 98), (98, 99), (

In [83]:
sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

In [85]:
for idx, _ in enumerate(sample_mapping):
    answer = sample_dataset["answers"][sample_mapping[idx]]     #sample_mapping[idx]找到这个窗口对应的样本索引（比如样本2）,变成sample_dataset["answers"][2],即answer对应 由10个样本token化成的sample_dataset里的  “answers”里的  样本2对应的内容
    start_char = answer["answer_start"][0]
    end_char = start_char + len(answer["text"][0])
    # 定位答案在token中的起始位置和结束位置
    # 一种策略，我们要拿到context的起始和结束，然后从左右两侧向答案逼近

    context_start = tokenized_examples.sequence_ids(idx).index(1)
    context_end = tokenized_examples.sequence_ids(idx).index(None, context_start) - 1

    offset = tokenized_examples.get("offset_mapping")[idx]
    example_ids = []

    # 判断答案是否在context中
    if offset[context_end][1] < start_char or offset[context_start][0] > end_char:
        start_token_pos = 0
        end_token_pos = 0
    else:
        token_id = context_start
        while token_id <= context_end and offset[token_id][0] < start_char:
            token_id += 1
        start_token_pos = token_id
        token_id = context_end
        while token_id >= context_start and offset[token_id][1] > end_char:
            token_id -=1
        end_token_pos = token_id
        example_ids.append([sample_mapping[idx]])

    print(answer, start_char, end_char, context_start, context_end, start_token_pos, end_token_pos)
    print("token answer decode:", tokenizer.decode(tokenized_examples["input_ids"][idx][start_token_pos: end_token_pos + 1]))

In [27]:
def process_func(examples):
    tokenized_examples = tokenizer(text=examples["question"],
                               text_pair=examples["context"],
                               return_offsets_mapping=True,
                               return_overflowing_tokens=True,
                               stride=128,
                               max_length=384, truncation="only_second", padding="max_length")
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    start_positions = []
    end_positions = []
    example_ids = []
    for idx, _ in enumerate(sample_mapping):
        answer = examples["answers"][sample_mapping[idx]]   # idx 是窗口索引，需要通过sample_mapping找到原始样本,
                                                            # sample_mapping[idx]找到这个窗口对应的样本索引（比如样本2）
        start_char = answer["answer_start"][0]
        end_char = start_char + len(answer["text"][0])
        # 定位答案在token中的起始位置和结束位置
        # 一种策略，我们要拿到context的起始和结束，然后从左右两侧向答案逼近
        context_start = tokenized_examples.sequence_ids(idx).index(1)
        context_end = tokenized_examples.sequence_ids(idx).index(None, context_start) - 1
        offset = tokenized_examples.get("offset_mapping")[idx]
        # 判断答案是否在context中
        if offset[context_end][1] < start_char or offset[context_start][0] > end_char:
            start_token_pos = 0
            end_token_pos = 0
        else:
            token_id = context_start
            while token_id <= context_end and offset[token_id][0] < start_char:
                token_id += 1
            start_token_pos = token_id
            token_id = context_end
            while token_id >= context_start and offset[token_id][1] > end_char:
                token_id -=1
            end_token_pos = token_id
        start_positions.append(start_token_pos)
        end_positions.append(end_token_pos)
        example_ids.append(examples["id"][sample_mapping[idx]])
        tokenized_examples["offset_mapping"][idx] = [
            (o if tokenized_examples.sequence_ids(idx)[k] == 1 else None)       #恒等于1代表为context值，否则就是CLS或者SEP，返回None
            for k, o in enumerate(tokenized_examples["offset_mapping"][idx])
        ]
        '''
        offset_mapping是元组列表，每个元组是 (start, end)，表示这个token在原始文本中从第start个字符到第end个字符（不包括end）
        sequence_ids:0：表示该token属于第一个序列（在MRC中就是问题）1：表示该token属于第二个序列（在MRC中就是上下文）
                    None：表示该token是特殊标记（如[CLS]、[SEP]、[PAD]等）

        offset_mapping对应的是字符位置映射，记录每个token在对应文本中的字符位置
        sample_mapping对应的是样本索引映射，记录每个窗口特征对应哪个原始样本


        k：当前token在序列中的索引位置 (0, 1, 2, ...)
        o：当前token对应的字符偏移元组 (start_char, end_char)

        处理前：⭐⭐⭐⭐⭐
        假设一个序列的token分布：
        tokens:      [CLS] 问题token1 问题token2 [SEP] 上下文token1 上下文token2 [SEP] [PAD]
        sequence_ids: [None, 0,        0,        None,  1,          1,          None, None]
        offset_mapping: [(0,1), (1,2), (2,3), (3,4), (4,5), (5,6), (6,7), (7,8)]

        处理后
        # 只保留上下文部分(sequence_id=1)的offset_mapping
        offset_mapping: [None, None, None, None, (4,5), (5,6), None, None]
        '''

    
    tokenized_examples["example_ids"] = example_ids
    tokenized_examples["start_positions"] = start_positions
    tokenized_examples["end_positions"] = end_positions
    return tokenized_examples

In [15]:
tokenied_datasets = datasets.map(process_func, batched=True, remove_columns=datasets["train"].column_names)
tokenied_datasets

Map:   0%|          | 0/10142 [00:00<?, ? examples/s]

Map:   0%|          | 0/3219 [00:00<?, ? examples/s]

Map:   0%|          | 0/1002 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_ids', 'start_positions', 'end_positions'],
        num_rows: 19189
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_ids', 'start_positions', 'end_positions'],
        num_rows: 6327
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_ids', 'start_positions', 'end_positions'],
        num_rows: 1988
    })
})

In [16]:
print(tokenied_datasets["train"]["offset_mapping"][1])

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, [266, 267], [267, 268], [268, 269], [269, 270], [270, 271], [271, 272], [272, 273], [273, 274], [274, 275], [275, 276], [276, 277], [277, 278], [278, 279], [279, 280], [280, 281], [281, 282], [282, 283], [283, 284], [284, 285], [285, 286], [286, 287], [287, 288], [288, 289], [289, 290], [290, 291], [291, 292], [292, 293], [293, 294], [294, 295], [295, 296], [296, 297], [297, 298], [298, 299], [299, 300], [300, 301], [301, 302], [302, 303], [303, 304], [304, 305], [305, 306], [306, 307], [307, 308], [308, 309], [309, 310], [310, 311], [311, 312], [312, 313], [313, 314], [314, 315], [315, 316], [316, 317], [317, 318], [318, 319], [319, 320], [320, 321], [321, 325], [325, 326], [326, 327], [327, 328], [328, 329], [329, 330], [330, 331], [331, 332], [332, 333], [333, 334], [334, 335], [335, 336], [336, 337], [337, 338], [338, 339], [339, 340], [340, 341], [341, 342], [342, 343], [343, 344

In [17]:
tokenied_datasets["train"]["example_ids"][:10]

['TRAIN_186_QUERY_0',
 'TRAIN_186_QUERY_0',
 'TRAIN_186_QUERY_0',
 'TRAIN_186_QUERY_1',
 'TRAIN_186_QUERY_1',
 'TRAIN_186_QUERY_1',
 'TRAIN_186_QUERY_2',
 'TRAIN_186_QUERY_2',
 'TRAIN_186_QUERY_2',
 'TRAIN_186_QUERY_3']

In [19]:
#建立原始样本(example)与特征窗口(feature)之间的反向映射关系
import collections

# example 和 feature的映射
example_to_feature = collections.defaultdict(list)
'''
defaultdict 的工作原理⭐⭐⭐⭐⭐
    # 如果是普通字典：访问不存在的键会报KeyError
    normal_dict = {}
    normal_dict["key"]  # ❌ KeyError

    # 使用defaultdict：访问不存在的键会自动创建默认值
    from collections import defaultdict
    default_dict = defaultdict(list)
    default_dict["key"]  # ✅ 返回空列表 []，并创建这个键
'''

for idx, example_id in enumerate(tokenied_datasets["train"]["example_ids"][:10]):       #遍历所有example_id和这个example_id对应的索引值
    example_to_feature[example_id].append(idx)
    #实现功能：如果遇到example_to_feature里面没有的example_id，则增加一个名为该example_id并添加其在原tokenied_datasets里面的索引值，如果有，不用创建直接添加其所对应的idx
example_to_feature      #打印这个新的字典example_id_feature
'''
正向映射：overflow_to_sample_mapping（特征 → 原始样本）
反向映射：example_to_feature（原始样本 → 所有相关特征）
'''

defaultdict(list,
            {'TRAIN_186_QUERY_0': [0, 1, 2],
             'TRAIN_186_QUERY_1': [3, 4, 5],
             'TRAIN_186_QUERY_2': [6, 7, 8],
             'TRAIN_186_QUERY_3': [9]})

## Step4 获取模型输出

In [20]:
import numpy as np
import collections

def get_result(start_logits, end_logits, exmaples, features):       #功能：将模型输出的 start_logits 和 end_logits 转换为实际的文本答案
    '''
    start_logits：模型预测的答案起始位置分数
    end_logits：模型预测的答案结束位置分数
    examples：原始样本数据
    features：预处理后的特征数据
    '''
    predictions = {}
    references = {}

    # example 和 feature的映射  为后续整合多个窗口的预测做准备
    example_to_feature = collections.defaultdict(list)
    for idx, example_id in enumerate(features["example_ids"]):
        example_to_feature[example_id].append(idx)

    # 最优答案候选(每个窗口考虑的候选答案数量)
    n_best = 20
    # 最大答案长度(答案最大长度限制)
    max_answer_length = 30

    for example in exmaples:
        example_id = example["id"]
        context = example["context"]
        answers = []        # 存储所有候选答案
        for feature_idx in example_to_feature[example_id]:
        #⭐⭐⭐⭐⭐feature_idx作为for i in里面的i，表示example_to_feature[example_id]中可迭代的对象，也就是example_id的索引值
            start_logit = start_logits[feature_idx]     #start_logit包含的是模型对当前窗口特征中每个token作为答案起始位置的概率分数。分数具体多少由后续模型自己输出，此处只是建一个分数列表，里面每一个start_logit并没实际值
            end_logit = end_logits[feature_idx]
            offset = features[feature_idx]["offset_mapping"]        # 获取当前窗口的偏移映射
            #offset可能看起来像：[None,None,None,(0,1),(1,2)....]
            start_indexes = np.argsort(start_logit)[::-1][:n_best].tolist()
            '''
            np.argsort(start_logit)[::-1]用于对 数组start_logit 进行降序排序，并返回排序后元素的索引。[:n_best]表示取前n_best个。.tolist()将numpy数组转换为Python列表
            #对数组start_logit 进行降序排序，并返回“用索引代替原分数进行排序”。并取前n_best个。再将这些取出的n_bset个转换为Python列表
            例子：
            start_logit = [ -5.2,  3.1, -2.8,  8.5,  1.2, -7.3]  # 6个token的分数           n_best = 3
            降序排序后的索引: [3, 1, 4, 2, 0, 5] (对应值: 8.5, 3.1, 1.2, -2.8, -5.2, -7.3)
            [:n_best] 取前3个    [3, 1, 4]，   意为模型认为最有可能成为答案起始位置的三个token的索引，这索引是当前窗口（start_logit的其中一个窗口）（当前窗口如何得来，看下面的数据流向）中token序列的索引
            tolist() 转换         start_indexes = [3, 1, 4]


            # ⭐⭐⭐⭐⭐数据流向：
            examples (原始数据集)
                → example["id"] (原始样本ID)
                    → example_to_feature[example_id] (该样本对应的窗口特征索引列表)
                         → start_logits[feature_idx] (某个窗口特征的起始位置分数)
                                → start_indexes (该窗口内最有可能的token索引)
            '''

            end_indexes = np.argsort(end_logit)[::-1][:n_best].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    '''情况1：最后可能作为答案结尾的token不在当前窗口中————如果offset里面最有可能成为答案的n_best个起始位置或结束位置是none'''
                    if offset[start_index] is None or offset[end_index] is None:
                        continue    #跳出当前这次for循环，重新执行下一个end_index的for循环
                    '''情况2：token在当前窗口中————逻辑顺序是否合理：如果最有可能成为答案结尾的token 小于 当前窗口内 最有可能作为答案开头的token
                                                答案长度是否合理：或者二者长度大于最大答案长度'''
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue
                    '''除上述两种情况外，就都是正确情况,记录候选答案'''
                    answers.append({
                        "text": context[offset[start_index][0]: offset[end_index][1]],
                        #注意：此处逻辑顺序是：文段context取的是“offset[start_index]的第0位，到，offset[end_index]的第1位”
                        "score": start_logit[start_index] + end_logit[end_index]
                    })

                    '此时，由于有很多组答案，所以answer可能长这样：answer={["text":"北京",  "score": 8.9],  ["text":"北京科技","score": 8.9],...}'
        #接下来对这么多组答案，选择分数最高的答案
        #⭐⭐⭐⭐⭐对于当前样本，可能出现遍历完整个start_indexes和end_indexes之后，都没有正确的情况（即所有index一直在情况1和2之间），所以进行长度判断
        if len(answers) > 0:    #如果有 有效答案
            best_answer = max(answers, key=lambda x: x["score"])
            predictions[example_id] = best_answer["text"]
        else:       #如果没有 有效答案
            predictions[example_id] = ""
        references[example_id] = example["answers"]["text"]

    return predictions, references

## Step5 评估函数

In [86]:
from cmrc_eval import evaluate_cmrc

def metirc(pred):
    #start_logits: 模型预测的每个token作为答案起始位置的分数矩阵
    start_logits, end_logits = pred[0]      #pred是一个元组，pred 的结构：([start_logits, end_logits], ...其他信息)
    #根据数据量判断当前是验证集还是测试集     判断逻辑：如果预测样本数等于验证集特征数 → 当前是验证集，否则是测试集
    if start_logits.shape[0] == len(tokenied_datasets["validation"]):   #start_logits.shape = (batch_size, sequence_length)
        p, r = get_result(start_logits, end_logits, datasets["validation"], tokenied_datasets["validation"])
    else:
        p, r = get_result(start_logits, end_logits, datasets["test"], tokenied_datasets["test"])
    return evaluate_cmrc(p, r)      #计算并返回评估指标
    '''
    参数：
    p (predictions)：模型预测的答案字典 {example_id: "预测答案", ...}
    r (references)：真实答案字典 {example_id: ["真实答案1", "真实答案2", ...], ...}
    '''

## Step6 加载模型

In [87]:
model = AutoModelForQuestionAnswering.from_pretrained("hfl/chinese-macbert-base")

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at hfl/chinese-macbert-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Step7 配置TrainingArguments

In [88]:
args = TrainingArguments(
    output_dir="models_for_qa",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    eval_strategy="steps",
    eval_steps=200,
    save_strategy="epoch",
    logging_steps=50,
    num_train_epochs=1
)

## Step8 配置Trainer

In [89]:
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenied_datasets["train"],
    eval_dataset=tokenied_datasets["validation"],
    data_collator=DefaultDataCollator(),
    compute_metrics=metirc
)

## Step9 模型训练

In [91]:
trainer.train()

Step,Training Loss,Validation Loss


KeyboardInterrupt: 

## Step8 模型预测

In [ ]:
from transformers import pipeline

pipe = pipeline("question-answering", model=model, tokenizer=tokenizer, device=0)
pipe

In [ ]:
pipe(question="小明在哪里上班？", context="小明在北京上班")